In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
from scipy import stats
import math as m
from sklearn.neighbors.kde import KernelDensity
from sympy import *
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm
from numpy import linspace, hstack
import requests 
from bs4 import BeautifulSoup
import urllib.request
import os
#from pyquery import PyQuery as pq
import pickle
from selenium import webdriver
import time
import calendar
import warnings
warnings.filterwarnings('ignore')


def compute_riskiness(data,i):
#    print data.var()/(2*data.mean())
    probDensityFun = gaussian_kde(data)
    x = linspace(-0.2,0.2,200)
    p=(probDensityFun(x)/probDensityFun(x).sum())
    g=list(map(lambda x: m.exp(-x*i), x))
    return 1-p.dot(g)

def compute_riskiness_r(data):
    if data.mean()<0:
        temp_past=1
        i=0
        while temp_past>=-1e-6 and compute_riskiness(data,i)>=-1e-6:
            temp_past=compute_riskiness(data,i)
            i=i-1
        i=i+1
        temp_past=1
        j=0
        while temp_past>=-1e-6 and compute_riskiness(data,i+j/10.0)>=-1e-6:
            temp_past=compute_riskiness(data,i+j/10.0)
            j=j-1
        j=j+1
        temp_past=1
        k=0
        while temp_past>=-1e-6 and compute_riskiness(data,i+j/10.0+k/100.0)>=-1e-6:
            temp_past=compute_riskiness(data,i+j/10.0+k/100.0)
            k=k-1
        k=k+1
        return i+j/10.0+k/100.0
    else:
        temp_past=1
        i=0
        while temp_past>=-1e-6 and compute_riskiness(data,i)>=-1e-6:
            temp_past=compute_riskiness(data,i)
            i=i+1
        i=i-1
        temp_past=1
        j=0
        while temp_past>=-1e-6 and compute_riskiness(data,i+j/10.0)>=-1e-6:
            temp_past=compute_riskiness(data,i+j/10.0)
            j=j+1
        j=j-1
        temp_past=1
        k=0
        while temp_past>=-1e-6 and compute_riskiness(data,i+j/10.0+k/100.0)>=-1e-6:
            temp_past=compute_riskiness(data,i+j/10.0+k/100.0)
            k=k+1
        k=k-1
        return i+j/10.0+k/100.0
    
def check_download_file(com_data):
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data_yahoo'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(options=options) 
    
    for s in Symbol:
        data = com_data[s].values
        if (np.sum(np.isnan(data)) > 10):
            os.remove(f'./data_yahoo/{s}.csv')
            date_time=Inception.loc[s]
            date_time=dt.datetime.strptime(date_time, '%d/%m/%Y').strftime("%m/%d/%Y")
            get_single_symbol(s,browser,date_time,5)
    browser.quit()
    
def get_single_symbol(symbol,browser,date,wait = 3):
    # check directory
    if not os.path.isdir('./data_yahoo'):
        os.mkdir('./data_yahoo')
    
    filepath = f'./data_yahoo/{symbol}.csv'
    if os.path.isfile(filepath):
        print('File exist')
        return 
    
    url = f'https://finance.yahoo.com/quote/{symbol}/history?p={symbol}'
    browser.get(url)
    browser.find_element_by_css_selector("[class='C(t) O(n):f Tsh($actionBlueTextShadow) Bd(n) Bgc(t) Fz(14px) Pos(r) T(-1px) Bd(n):f Bxsh(n):f Cur(p) W(190px)']").click()
    start_date = browser.find_element_by_css_selector("[class='Bdrs(0) Bxsh(n)! Fz(s) Bxz(bb) D(ib) Bg(n) Pend(5px) Px(8px) Py(0) H(34px) Lh(34px) Bd O(n):f O(n):h Bdc($c-fuji-grey-c) Bdc($c-fuji-blue-1-b):f M(0) Pstart(10px) Bgc(white) W(90px) Mt(5px)']")
    start_date.clear()
    start_date.send_keys(date)
    end_date = browser.find_element_by_css_selector("[class='Bdrs(0) Bxsh(n)! Fz(s) Bxz(bb) D(ib) Bg(n) Pend(5px) Px(8px) Py(0) H(34px) Lh(34px) Bd O(n):f O(n):h Bdc($c-fuji-grey-c) Bdc($c-fuji-blue-1-b):f M(0) Pstart(10px) Bgc(white) W(90px) Mt(5px) Mstart(15px)']")
    end_date.clear()
    end_date.send_keys(today)

    try:
        browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Miw(80px)! Fl(start)']").click()
    except:
        browser.find_element_by_css_selector("[class=' Bd Bdc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) D(ib) C($c-fuji-blue-1-b) Bdc($actionBlueHover):h C($actionBlueHover):h Cur(p) Td(n)  Py(8px) cancel Miw(80px)! Fl(end)']").click()
        browser.find_element_by_css_selector("[class='C(t) O(n):f Tsh($actionBlueTextShadow) Bd(n) Bgc(t) Fz(14px) Pos(r) T(-1px) Bd(n):f Bxsh(n):f Cur(p) W(190px)']").click()
        browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Miw(80px)! Fl(start)']").click()

    browser.find_element_by_css_selector("[class=' Bgc($c-fuji-blue-1-b) Bdrs(3px) Px(20px) Miw(100px) Whs(nw) Fz(s) Fw(500) C(white) Bgc($actionBlueHover):h Bd(0) D(ib) Cur(p) Td(n)  Py(9px) Fl(end)']").click()
    time.sleep(5)
    browser.find_element_by_css_selector("[class='Fl(end) Mt(3px) Cur(p)']").click()
    time.sleep(wait)

    while not os.path.isfile(filepath):
        browser.find_element_by_css_selector("[class='Fl(end) Mt(3px) Cur(p)']").click()
        time.sleep(wait)

    print('Download',symbol)
    
def get_all_symbol_csv(dic):
    
    # check directory
    if not os.path.isdir('./data_yahoo'):
        os.mkdir('./data_yahoo')

    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data_yahoo'}
    options.add_experimental_option('prefs', prefs)
    browser = webdriver.Chrome(options=options) 
    
    for s in Symbol:
        try:
            date_time = Inception.loc[s]
            date_time = dt.datetime.strptime(date_time, '%d/%m/%Y').strftime("%m/%d/%Y")
            get_single_symbol(s,browser,date_time)
        except:
            continue

    browser.quit()
    
def combine_data(dic,last_day_of_month):
    com_data = pd.DataFrame(last_day_of_month[::-1],columns=['Date'])
        
    for s in Symbol:
        data = pd.read_csv(f'./data_yahoo/{s}.csv')
        data['Date'] = pd.to_datetime(data['Date'], format="%Y-%m-%d")
        
        # Mask
        mask = [False for i in range(data['Date'].shape[0])]
        for day in last_day_of_month:
            temp = data['Date'] == day
            mask |= temp
        data = data[mask]
        
        data_close = data[['Date','Close']].reset_index(drop=True)
        data_close = data_close.rename(columns={'Close':s})
        com_data = com_data.merge(data_close,on='Date',how='left')
        
    # check directory
    if not os.path.isdir('./Combine_data'):
        os.mkdir('./Combine_data')
    com_data.to_csv('./Combine_data/com_data.csv')
    
    return com_data

def combine_daily_data(Symbol):
    com_data = pd.DataFrame()
    for s in Symbol:
        data = pd.read_csv(f'./data_yahoo/{s}.csv',index_col=0,parse_dates=True)["Adj Close"]
        com_data = pd.concat([com_data,data],axis=1)
    com_data.columns = Symbol
    return com_data

def check_download_file(com_data):
    options = webdriver.ChromeOptions()
    prefs = {'download.default_directory': './data_yahoo'}
    options.add_experimental_option('prefs', prefs)
    browser=webdriver.Chrome(options=options) 
    
    for s in Symbol:
        data = com_data[s].values
        if (np.sum(np.isnan(data)) > 10):
            os.remove(f'./data_yahoo/{s}.csv')
            get_single_symbol(s,browser,5)
    browser.quit()

In [ ]:
# USG3M.xlsx檔案裡有三組資料，但因為沒有head，看不懂是什麼資料，似乎與無風險利率有關
bond_rate = pd.read_excel("USG3M.xlsx",index_col=0,parse_dates=True,header=None)

# 取第一組資料
USG3M = bond_rate[1]

Inception = pd.read_excel("origin_list.xlsm",index_col=0)["Inception"]
# 取出ETF名稱(簡寫)
Symbol = list(Inception.index)
# 取得今天日期 (月/日/年)
today = str(dt.datetime.today().month)+"/"+str(dt.datetime.today().day)+"/"+str(dt.datetime.today().year)

# ***** 從Yahoo下載ETF資料(爬蟲)，不用執行 *****
#get_all_symbol_csv(Symbol)

# 把所有ETF資料合在一個dataframe
etf_data = combine_daily_data(Symbol)
etf_data.head()

In [ ]:
day = etf_data.index[0]
index_num = 0
while day != etf_data.index[-1]:
    if (etf_data.index[index_num+1]-etf_data.index[index_num]).days != 1:
        for i in range(1,(etf_data.index[index_num+1]-etf_data.index[index_num]).days):
            etf_data.loc[etf_data.index[index_num]+dt.timedelta(i)] = etf_data.loc[etf_data.index[index_num]]
    etf_data = etf_data.sort_index()
    index_num = index_num + (etf_data.index[index_num+1] - etf_data.index[index_num]).days
    day = etf_data.index[index_num]

In [ ]:
remove_columns = []
for i in range(etf_data.shape[1]):
    try:
        etf_data.iloc[:,i].dropna().astype(float)
    except:
        remove_columns.append(etf_data.columns[i])
for x in remove_columns:
    etf_data.pop(x)

In [ ]:
# 取出週五的index
week_index = []
for i in range(etf_data.shape[0]):
    if etf_data.index[i].isoweekday()==5:
        week_index.append(etf_data.index[i])
week_index = week_index[:-1]

In [ ]:
# 取出每月1號index
month_index = []
for i in range(etf_data.shape[0]):
    if etf_data.index[i].day==1:
        month_index.append(etf_data.index[i])

In [ ]:
etf_data_week = etf_data.loc[week_index]
etf_data_month = etf_data.loc[month_index]
#etf_data_week.to_csv("etf_data_week.csv")
#etf_data_month.to_csv("etf_data_month.csv")
print(etf_data_week.shape)
print(etf_data_month.shape)

In [ ]:
#etf_data_week=pd.read_csv("etf_data_week.csv",index_col=0,parse_dates=True)
#etf_data_month=pd.read_csv("etf_data_month.csv",index_col=0,parse_dates=True)

In [ ]:
etf_data_week_return = etf_data_week/etf_data_week.shift(1)
etf_data_month_return = etf_data_month/etf_data_month.shift(1)
print(etf_data_week_return.shape)
print(etf_data_month_return.shape)

### 以下7段是取EWM這支ETF來畫圖，可以跳過

In [ ]:
data_for_plot_week = etf_data_week_return["EWM"][1:]
data_for_plot_month = etf_data_month_return["EWM"][1:]

In [ ]:
plt.figure(figsize=(10,10))
plt.hist(data_for_plot_week,bins='auto', color='#0504aa',alpha=0.7, rwidth=0.85)
plt.show()
plt.figure(figsize=(10,10))
sns.set_style('darkgrid')
sns.distplot(data_for_plot_week)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.hist(data_for_plot_month,bins='auto', color='#0504aa',alpha=0.7, rwidth=0.85)
plt.show()
plt.figure(figsize=(10,10))
sns.set_style('darkgrid')
sns.distplot(data_for_plot_month)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10))
res = stats.probplot(data_for_plot_week,dist="norm", plot=plt)
plt.title("Probplot for normal")
plt.show()
fig = plt.figure(figsize=(10,10))
res = stats.probplot(np.log(data_for_plot_week),dist="norm", plot=plt)
plt.title("Probplot for normal")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10))
res = stats.probplot(data_for_plot_month,dist="norm", plot=plt)
plt.title("Probplot for normal")
plt.show()
fig = plt.figure(figsize=(10,10))
res = stats.probplot(np.log(data_for_plot_month),dist="norm", plot=plt)
plt.title("Probplot for normal")
plt.show()

In [ ]:
temp=[]
for i in range(-1000,1000):
    temp.append(compute_riskiness(data_for_plot.iloc[0:156]-1,i/100.0))

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(range(-1000,1000),temp)
plt.show()

### omega 週資料

In [ ]:
# 每週有一個omega, why?
omega_combine_week = pd.DataFrame()
columns_name = []

for k in range(etf_data_week_return.shape[1]):
    print(k)
    temp_etf = etf_data_week_return.iloc[:,k].dropna()
    omega = []
    if len(temp_etf) >= 156:
        columns_name.append(etf_data_week_return.columns[k])
    for i in range(156,len(temp_etf)+1):
        # 機率密度
        probDensityFun = gaussian_kde(temp_etf.iloc[i-156:i]-1)
        x = np.linspace(-0.5,0.5,10000)
        # 無風險利率
        L = USG3M.loc[temp_etf.iloc[i-156:i].index[-1]]
        p = (probDensityFun(x)/probDensityFun(x).sum())
        p = p.cumsum()
        p = pd.DataFrame(p,index=x)
        omega.append(((1-p).loc[L/100/52:].sum())/(p.loc[:L/100/52].sum()))
    omega = pd.DataFrame(omega,index=temp_etf.index[155:])
    omega_combine_week = pd.concat([omega_combine_week,omega],axis=1)
omega_combine_week.columns = columns_name

### omega月資料

In [ ]:
omega_combine_month = pd.DataFrame()
columns_name = []
for k in range(etf_data_month_return.shape[1]):
    print(k)
    temp_etf=etf_data_month_return.iloc[:,k].dropna()
    omega=[]
    if len(temp_etf)>=36:
        columns_name.append(etf_data_month_return.columns[k])
    for i in range(36,len(temp_etf)+1):
        probDensityFun = gaussian_kde(temp_etf.iloc[i-36:i]-1)
        x=np.linspace(-0.5,0.5,10000)
        L=USG3M.loc[temp_etf.iloc[i-36:i].index[-1]]
        p=(probDensityFun(x)/probDensityFun(x).sum())
        p=p.cumsum()
        p=pd.DataFrame(p,index=x)
        omega.append(((1-p).loc[L/100/12:].sum())/(p.loc[:L/100/12].sum()))
    omega=pd.DataFrame(omega,index=temp_etf.index[35:])
    omega_combine_month=pd.concat([omega_combine_month,omega],axis=1)
omega_combine_month.columns=columns_name

### riskiness r週資料

In [ ]:
riskiness_combine_week=pd.DataFrame()
columns_name=[]
for k in range(etf_data_week_return.shape[1]):
    print(k)
    temp_etf=etf_data_week_return.iloc[:,k].dropna()
    riskiness=[]
    if len(temp_etf)>=156:
        columns_name.append(etf_data_week_return.columns[k])
        for i in range(156,len(temp_etf)+1):
            riskiness.append(compute_riskiness_r(temp_etf.iloc[i-156:i]-1))
        riskiness=pd.DataFrame(riskiness,index=temp_etf.index[155:])
        riskiness=riskiness[0].map(lambda x: m.exp(-x))
        riskiness=pd.DataFrame(riskiness,index=temp_etf.index[155:])
        riskiness_combine_week=pd.concat([riskiness_combine_week,riskiness],axis=1)
riskiness_combine_week.columns=columns_name

### riskiness r月資料

In [ ]:
riskiness_combine_month=pd.DataFrame()
columns_name=[]
for k in range(etf_data_month_return.shape[1]):
    print(k)
    temp_etf=etf_data_month_return.iloc[:,k].dropna()
    riskiness=[]
    if len(temp_etf)>=36:
        columns_name.append(etf_data_month_return.columns[k])
        for i in range(36,len(temp_etf)+1):
            riskiness.append(compute_riskiness_r(temp_etf.iloc[i-36:i]-1))
        riskiness=pd.DataFrame(riskiness,index=temp_etf.index[35:])
        riskiness=riskiness[0].map(lambda x: m.exp(-x))
        riskiness=pd.DataFrame(riskiness,index=temp_etf.index[35:])
        riskiness_combine_month=pd.concat([riskiness_combine_month,riskiness],axis=1)
riskiness_combine_month.columns=columns_name

### 存檔

In [ ]:
omega_combine_week.to_csv("omega_combine_week.csv")
omega_combine_month.to_csv("omega_combine_month.csv")
riskiness_combine_week.to_csv("riskiness_combine_week.csv")
riskiness_combine_month.to_csv("riskiness_combine_month.csv")

### adjust sharp ratio週資料

In [ ]:
adj_combine_week = pd.DataFrame()
cumprod_combine_week=pd.DataFrame()
columns_name=[]
for k in range(etf_data_week_return.shape[1]):
    temp_etf=etf_data_week_return.iloc[:,k].dropna()
    temp_etf=np.log(temp_etf)
    adj_var=[]
    cumprod_return=[]
    if len(temp_etf)>=156:
        columns_name.append(etf_data_week_return.columns[k])
        for i in range(156,len(temp_etf)+1):
            mean=(temp_etf.iloc[i-156:i].mean())
            std=temp_etf.iloc[i-156:i].std()
            skew=temp_etf.iloc[i-156:i].skew()
            kurt=temp_etf.iloc[i-156:i].kurt()
            var=-1.96
            Z=var+(skew*((var**2)-1)/6)+(kurt*((var**3)-3*var)/24)-((skew**2)*(2*(var**3)-5*var)/36)
            adj_var.append((-0.5)*(std**2)*52-Z*std*m.sqrt(156))
            cumprod_return.append(np.exp(temp_etf.iloc[i-156:i]).cumprod()[-1])
        adj_var=pd.DataFrame(adj_var,index=temp_etf.index[155:])
        adj_combine_week=pd.concat([adj_combine_week,adj_var],axis=1)
        cumprod_return=pd.DataFrame(cumprod_return,index=temp_etf.index[155:])
        cumprod_combine_week=pd.concat([cumprod_combine_week,cumprod_return],axis=1)
adj_combine_week.columns=columns_name
cumprod_combine_week.columns=columns_name
sharpe_adj_combine_week=cumprod_combine_week/adj_combine_week

### adjust sharp ratio月資料

In [ ]:
adj_combine_month=pd.DataFrame()
cumprod_combine_month=pd.DataFrame()
columns_name=[]
for k in range(etf_data_month_return.shape[1]):
    temp_etf=etf_data_month_return.iloc[:,k].dropna()
    temp_etf=np.log(temp_etf)
    adj_var=[]
    cumprod_return=[]
    if len(temp_etf)>=36:
        columns_name.append(etf_data_month_return.columns[k])
        for i in range(36,len(temp_etf)+1):
            mean=(temp_etf.iloc[i-36:i].mean())
            std=temp_etf.iloc[i-36:i].std()
            skew=temp_etf.iloc[i-36:i].skew()
            kurt=temp_etf.iloc[i-36:i].kurt()
            var=-1.96
            Z=var+(skew*((var**2)-1)/6)+(kurt*((var**3)-3*var)/24)-((skew**2)*(2*(var**3)-5*var)/36)
            adj_var.append((-0.5)*(std**2)*52-Z*std*m.sqrt(36))
            cumprod_return.append(np.exp(temp_etf.iloc[i-36:i]).cumprod()[-1])
        adj_var=pd.DataFrame(adj_var,index=temp_etf.index[35:])
        adj_combine_month=pd.concat([adj_combine_month,adj_var],axis=1)
        cumprod_return=pd.DataFrame(cumprod_return,index=temp_etf.index[35:])
        cumprod_combine_month=pd.concat([cumprod_combine_month,cumprod_return],axis=1)
adj_combine_month.columns=columns_name
cumprod_combine_month.columns=columns_name
sharpe_adj_combine_month=cumprod_combine_month/adj_combine_month

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(adj_combine_week["EWM"])
plt.plot(pd.rolling_std(data_for_plot,window=156)*m.sqrt(52),color='y')
#ax1=plt.twinx()
#ax1.plot(riskiness_combine["EWM"],color='r')
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(etf_data["EWM"])
plt.title('EWM')
ax1=plt.twinx()
ax1.plot(sharpe_adj_combine_week["EWM"],color='y')
ax1.plot(omega_combine_week["EWM"],color='r')
ax2=plt.twinx()
ax2.plot(riskiness_combine_week["EWM"].map(lambda x: -log(x)),color='g')
plt.show()

### 排序

In [ ]:
omega_combine_week_rank=omega_combine_week.rank(axis=1)
riskiness_combine_week_rank=riskiness_combine_week.rank(ascending=False,axis=1)
sharpe_adj_combine_week_rank=sharpe_adj_combine_week.rank(axis=1)
adj_combine_week_rank=adj_combine_week.rank(ascending=False,axis=1)

In [ ]:
omega_combine_month_rank=omega_combine_month.rank(axis=1)
riskiness_combine_month_rank=riskiness_combine_month.rank(ascending=False,axis=1)
sharpe_adj_combine_month_rank=sharpe_adj_combine_month.rank(axis=1)
adj_combine_month_rank=adj_combine_month.rank(ascending=False,axis=1)

In [ ]:
weight_omega_week=pd.DataFrame(columns=omega_combine_week_rank.columns)
for i in range(omega_combine_week.shape[0]):
    date_now=omega_combine_week.index[i]
    weight_omega_week.loc[date_now]=0
    weight_omega_week.loc[date_now][omega_combine_week_rank.iloc[i,:].dropna().sort_values()[-10:].index]=0.1
weight_riskiness_week=pd.DataFrame(columns=riskiness_combine_week_rank.columns)
for i in range(riskiness_combine_week.shape[0]):
    date_now=riskiness_combine_week.index[i]
    weight_riskiness_week.loc[date_now]=0
    weight_riskiness_week.loc[date_now][riskiness_combine_week_rank.iloc[i,:].dropna().sort_values()[-10:].index]=0.1
weight_sharpe_week=pd.DataFrame(columns=sharpe_adj_combine_week_rank.columns)
for i in range(sharpe_adj_combine_week.shape[0]):
    date_now=sharpe_adj_combine_week_rank.index[i]
    weight_sharpe_week.loc[date_now]=0
    weight_sharpe_week.loc[date_now][sharpe_adj_combine_week_rank.iloc[i,:].dropna().sort_values()[-10:].index]=0.1

In [ ]:
weight_omega_month=pd.DataFrame(columns=omega_combine_month_rank.columns)
for i in range(omega_combine_month.shape[0]):
    date_now=omega_combine_month.index[i]
    weight_omega_month.loc[date_now]=0
    weight_omega_month.loc[date_now][omega_combine_month_rank.iloc[i,:].dropna().sort_values()[-10:].index]=0.1
weight_riskiness_month=pd.DataFrame(columns=riskiness_combine_month_rank.columns)
for i in range(riskiness_combine_month.shape[0]):
    date_now=riskiness_combine_month.index[i]
    weight_riskiness_month.loc[date_now]=0
    weight_riskiness_month.loc[date_now][riskiness_combine_month_rank.iloc[i,:].dropna().sort_values()[-10:].index]=0.1
weight_sharpe_month=pd.DataFrame(columns=sharpe_adj_combine_month_rank.columns)
for i in range(sharpe_adj_combine_month.shape[0]):
    date_now=sharpe_adj_combine_month_rank.index[i]
    weight_sharpe_month.loc[date_now]=0
    weight_sharpe_month.loc[date_now][sharpe_adj_combine_month_rank.iloc[i,:].dropna().sort_values()[-10:].index]=0.1

In [ ]:
enough_index_week=omega_combine_week_rank[omega_combine_week_rank.max(axis=1)>30].index
enough_index_month=omega_combine_month_rank[omega_combine_month_rank.max(axis=1)>30].index

In [ ]:
allocation_omega_week=((weight_omega_week.shift(1))*etf_data_week_return).sum(axis=1)[enough_index_week]
allocation_riskiness_week=((weight_riskiness_week.shift(1))*etf_data_week_return).sum(axis=1)[enough_index_week]
allocation_sharpe_week=((weight_sharpe_week.shift(1))*etf_data_week_return).sum(axis=1)[enough_index_week]
allocation_omega_week_price=allocation_omega_week.cumprod()
allocation_riskiness_week_price=allocation_riskiness_week.cumprod()
allocation_sharpe_week_price=allocation_sharpe_week.cumprod()

In [ ]:
allocation_omega_month=((weight_omega_month.shift(1))*etf_data_month_return).sum(axis=1)[enough_index_month]
allocation_riskiness_month=((weight_riskiness_month.shift(1))*etf_data_month_return).sum(axis=1)[enough_index_month]
allocation_sharpe_month=((weight_sharpe_month.shift(1))*etf_data_month_return).sum(axis=1)[enough_index_month]
allocation_omega_month_price=allocation_omega_month.cumprod()
allocation_riskiness_month_price=allocation_riskiness_month.cumprod()
allocation_sharpe_month_price=allocation_sharpe_month.cumprod()

In [ ]:
average_corr_week=[]
for i in range(omega_combine_week_rank.shape[1]):
    average_corr_week.append((pd.concat([omega_combine_week_rank.iloc[:,i].dropna(),riskiness_combine_week_rank.iloc[:,i].dropna(),sharpe_adj_combine_week_rank.iloc[:,i].dropna()],axis=1).corr().sum().sum()-3)/6)
print(np.array(average_corr_week).mean())

In [ ]:
average_corr_month=[]
for i in range(omega_combine_month_rank.shape[1]):
    average_corr_month.append((pd.concat([omega_combine_month_rank.iloc[:,i].dropna(),riskiness_combine_month_rank.iloc[:,i].dropna(),sharpe_adj_combine_month_rank.iloc[:,i].dropna()],axis=1).corr().sum().sum()-3)/6)
print(np.array(average_corr_month).mean())

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(average_corr_week,label='week')
plt.plot(average_corr_month,label='month')
plt.legend()
plt.show()

In [ ]:
add_rank_week=omega_combine_week_rank+riskiness_combine_week_rank+sharpe_adj_combine_week_rank
add_rank_month=omega_combine_month_rank+riskiness_combine_month_rank+sharpe_adj_combine_month_rank

In [ ]:
weight_week=pd.DataFrame(columns=add_rank_week.columns)
for i in range(add_rank_week.shape[0]):
    date_now=add_rank_week.index[i]
    weight_week.loc[date_now]=0
    weight_week.loc[date_now][add_rank_week.iloc[i,:].dropna().sort_values()[-10:].index]=0.1
allocation_week=((weight_week.shift(1))*etf_data_week_return).sum(axis=1)[enough_index_week]

In [ ]:
weight_month=pd.DataFrame(columns=add_rank_month.columns)
for i in range(add_rank_month.shape[0]):
    date_now=add_rank_month.index[i]
    weight_month.loc[date_now]=0
    weight_month.loc[date_now][add_rank_month.iloc[i,:].dropna().sort_values()[-10:].index]=0.1
allocation_month=((weight_month.shift(1))*etf_data_month_return).sum(axis=1)[enough_index_month]

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(allocation_omega_week_price,label='omega')
plt.plot(allocation_riskiness_week_price,label='riskiness')
plt.plot(allocation_sharpe_week_price,label='sharpe')
plt.plot(allocation_week.cumprod(),label='add')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(allocation_omega_month_price,label='omega')
plt.plot(allocation_riskiness_month_price,label='riskiness')
plt.plot(allocation_sharpe_month_price,label='sharpe')
plt.plot(allocation_month.cumprod(),label='add')
plt.legend()
plt.show()

In [ ]:
plt.style.use('ggplot')
a=(allocation_omega_month_price[-1]**(12.0/allocation_omega_month_price.shape[0])-1)*100
b=allocation_omega_month.std()*365
c=a/b
e=(allocation_riskiness_month_price[-1]**(12.0/allocation_riskiness_month_price.shape[0])-1)*100
f=allocation_riskiness_month.std()*365
g=e/f
h=(allocation_sharpe_month_price[-1]**(12.0/allocation_sharpe_month_price.shape[0])-1)*100
i=allocation_sharpe_month.std()*365
j=h/i
a='%s'%  (np.round(a,2))+'%'
b='%s'%  (np.round(b,2))+'%'
c='%s'%  (np.round(c,2))
e='%s'%  (np.round(e,2))+'%'
f='%s'%  (np.round(f,2))+'%'
g='%s'%  (np.round(g,2))
h='%s'%  (np.round(h,2))+'%'
i='%s'%  (np.round(i,2))+'%'
j='%s'%  (np.round(j,2))
time=allocation_omega_month.index
x=allocation_omega_month_price[0]
drawdown_omega=min(allocation_omega_month_price[0]-1,0)
for i in range(1,len(allocation_omega_month_price)):
    if allocation_omega_month_price[i]>x:
        x=allocation_omega_month_price[i]
    drawdown_omega=np.append(drawdown_omega,-(1-allocation_omega_month_price[i]/x))
x=allocation_riskiness_month_price[0]
drawdown_riskiness=min(allocation_riskiness_month_price[0]-1,0)
for i in range(1,len(allocation_riskiness_month_price)):
    if allocation_riskiness_month_price[i]>x:
        x=allocation_riskiness_month_price[i]
    drawdown_riskiness=np.append(drawdown_riskiness,-(1-allocation_riskiness_month_price[i]/x))
x=allocation_sharpe_month_price[0]
drawdown_sharpe=min(allocation_sharpe_month_price[0]-1,0)
for i in range(1,len(allocation_sharpe_month_price)):
    if allocation_sharpe_month_price[i]>x:
        x=allocation_sharpe_month_price[i]
    drawdown_sharpe=np.append(drawdown_sharpe,-(1-allocation_sharpe_month_price[i]/x))
k=pd.DataFrame(drawdown_omega).min().iloc[0]*-100
l=pd.DataFrame(drawdown_riskiness).min().iloc[0]*-100
m=pd.DataFrame(drawdown_sharpe).min().iloc[0]*-100
k='%s'%  (np.round(k,2))+'%'
l='%s'%  (np.round(l,2))+'%'
m='%s'%  (np.round(m,2))+'%'
fig = plt.figure(figsize=(20,10))
plt.plot(time,np.log(allocation_omega_month_price),label='omega')
plt.plot(time,np.log(allocation_riskiness_month_price),label='riskiness')
plt.plot(time,np.log(allocation_sharpe_month_price),label='riskiness')
plt.legend()
col_labels=['return','std','sharp','max_drawdown']
row_labels=['omega','riskiness','sharpe']
table_vals=[[a,b,c,k],[e,f,g,l],[h,i,j,m]]
plt.table(cellText=table_vals,rowLabels=row_labels,colLabels=col_labels,loc='lower right',colWidths = [0.15]*4)
plt.show()
fig = plt.figure(figsize=(15,10))
ax1 = fig.add_subplot(3, 1, 1)
ax2 = fig.add_subplot(3, 1, 2)
ax3 = fig.add_subplot(3, 1, 3)
ax1.fill_between(time,0,drawdown_omega,facecolor='red')
ax1.set_ylim(ymax = 0, ymin = -0.4)
ax1.legend()
ax2.fill_between(time,0,drawdown_riskiness,facecolor='red')
ax2.set_ylim(ymax = 0, ymin = -0.4)
ax2.legend()
ax3.fill_between(time,0,drawdown_sharpe,facecolor='red')
ax3.set_ylim(ymax = 0, ymin = -0.4)
ax3.legend()
plt.show()